# Text embedding models
Embeddings类是一个设计用于与文本嵌入模型交互的类。有很多嵌入模型提供程序(OpenAI, coherence, hug Face等)，这个类的目的是为它们提供一个标准接口。

嵌入创建一段文本的矢量表示。这很有用，因为这意味着我们可以考虑向量空间中的文本，并进行语义搜索，在向量空间中寻找最相似的文本片段。

LangChain中的基Embeddings类提供了两个方法:一个用于嵌入文档，另一个用于嵌入查询。前者接受多个文本作为输入，后者接受单个文本作为输入。将它们作为两个独立的方法的原因是，一些嵌入提供程序对文档(要搜索的)和查询(搜索查询本身)具有不同的嵌入方法。

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug

load_dotenv(find_dotenv())
set_debug(False)

## Get started
首先，我们需要安装OpenAI合作伙伴包



In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

**embed_documents**

In [3]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

**embed_query**    
嵌入单查询      
为了与其他嵌入的文本进行比较而嵌入一段文本。  

In [4]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.005384807424727807,
 -0.0005522561790177147,
 0.03896066510130955,
 -0.002939867294003909,
 -0.008987877434176603]

## Caching
嵌入可以存储或临时缓存，以避免需要重新计算它们。

缓存嵌入可以使用`cachebackdebeddings`来完成。缓存支持的嵌入器是围绕在键值存储中缓存嵌入的嵌入器的包装器。文本被散列，散列用作缓存中的键。

初始化`cachebackdebeddings`的主要支持方式是从字节存储。它接受以下参数
- `underlying_embedder`:用于嵌入的嵌入器。
- `document_embedding_cache`:用于缓存文档嵌入的任何字节存储。
- `batch_size`:(可选，默认为None)在存储更新之间嵌入的文档数量。
- `namespace`:(可选，默认为"")用于文档缓存的命名空间。此命名空间用于避免与其他缓存冲突。例如，将其设置为所使用的嵌入模型的名称。

attention:
- 一定要设置名称`namespace`，以避免使用不同嵌入模型嵌入的相同文本发生冲突。
- 目前，`cachebackdebeddings`不缓存由`embed_query()`和`embed_query()`方法创建的嵌入。




In [5]:
from langchain.embeddings import CacheBackedEmbeddings

### Using with a Vector Store


In [6]:
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

underlying_embeddings = OpenAIEmbeddings()

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)
list(store.yield_keys())

[]

In [7]:
raw_documents = TextLoader("data/whatsapp_chat.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [8]:
%%time
db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 48 ms, sys: 72.9 ms, total: 121 ms
Wall time: 3.08 s


In [9]:
%%time
db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 3.21 ms, sys: 4.71 ms, total: 7.92 ms
Wall time: 8.16 ms


In [10]:
list(store.yield_keys())[:5]

['text-embedding-ada-00270f0ad1f-9b93-5ed0-9a76-26e7fac958fc']

### Swapping the ByteStore
为了使用不同的`ByteStore`，只需在创建`cachebackdebeddings`时使用它。下面，我们创建了一个等价的缓存嵌入对象，只不过使用了非持久化的`InMemoryByteStore`

In [11]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)